In [1]:
from proc_v1 import p, target, X_cat, X_cat2, X_num, X_all, sc
from ml_v2 import config, xgb_adapter, lgb_adapter, cb_adapter, nn_adapter, lr_adapter, scheduler, mapk_prob, ss, skf, train_data_proc
from sklearn.preprocessing import LabelEncoder
from functools import partial
import sgml
import numpy as np
import pandas as pd
le = LabelEncoder()
target_l = target + '_l'
df_train = p.fit_transform(['data/train.csv'])
df_org = p.transform(['data/Fertilizer Prediction.csv'])
df_test = p.transform(['data/test.csv'])
df_train = df_train.assign(
    **{target_l: lambda x: le.fit_transform(x[target])}
)
df_org = df_org.assign(
    **{target_l: lambda x: le.transform(x[target])}
)

config['train_data_proc'] = partial(train_data_proc, df_org = df_org)

2025-06-12 22:33:00.847328: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-12 22:33:00.856635: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749735180.867295   18758 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749735180.870632   18758 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1749735180.878918   18758 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking 

In [2]:
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
df_train[target].value_counts()

Fertilizer_Name
14-35-14    114436
10-26-26    113887
17-17-17    112453
28-28       111158
20-20       110889
DAP          94860
Urea         92317
Name: count, dtype: int64

In [17]:
df_train.pivot_table(
    index = target, columns = 'Crop_Type', values = 'Temparature', observed = True
)

Crop_Type,Sugarcane,Millets,Barley,Paddy,Pulses,Tobacco,Ground Nuts,Maize,Cotton,Wheat,Oil seeds
Fertilizer_Name,,,,,,,,,,,
28-28,31.531509,31.606595,31.499690,31.460042,31.502648,31.434453,31.546475,31.598354,31.525248,31.409131,31.611897
17-17-17,31.325002,31.513166,31.619634,31.552721,31.432562,31.515313,31.582805,31.518688,31.213795,31.501339,31.327225
10-26-26,31.435811,31.510430,31.460040,31.410943,31.337634,31.376449,31.587873,31.576825,31.572031,31.469278,31.503354
DAP,31.563676,31.595238,31.449926,31.586912,31.370627,31.424283,31.406645,31.601399,31.613239,31.433886,31.509260
20-20,31.460749,31.614611,31.469895,31.524957,31.404868,31.463014,31.618859,31.510853,31.479976,31.558386,31.590718
14-35-14,31.558982,31.553107,31.556938,31.575053,31.499388,31.524015,31.638489,31.559606,31.541344,31.447261,31.524108
Urea,31.475768,31.569604,31.408522,31.596824,31.399173,31.348242,31.662195,31.513423,31.506560,31.446550,31.657351


In [5]:
df_train.loc[df_train[target].isin(['28-28', '20-20'])]

,Temparature,Humidity,Moisture,Soil_Type,Crop_Type,Nitrogen,Potassium,Phosphorous,Fertilizer_Name,Crop_Soil,Fertilizer_Name_l
id,,,,,,,,,,,
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28,SugarcaneClayey,4
1,27,69,65,Sandy,Millets,30,6,18,28-28,MilletsSandy,4
5,30,59,29,Red,Pulses,10,0,9,20-20,PulsesRed,3
6,27,62,53,Sandy,Paddy,26,15,22,28-28,PaddySandy,4
9,28,50,35,Red,Tobacco,25,12,16,20-20,TobaccoRed,3
...,...,...,...,...,...,...,...,...,...,...,...
749990,36,50,61,Loamy,Paddy,4,10,0,28-28,PaddyLoamy,4
749991,26,67,58,Sandy,Pulses,32,15,4,28-28,PulsesSandy,4
749992,37,54,53,Red,Wheat,14,4,18,20-20,WheatRed,3


In [11]:
df_train.loc[df_train[target].isin(['28-28', '20-20'])]

,Temparature,Humidity,Moisture,Soil_Type,Crop_Type,Nitrogen,Potassium,Phosphorous,Fertilizer_Name,Crop_Soil,Fertilizer_Name_l
id,,,,,,,,,,,
0,37,70,36,Clayey,Sugarcane,36,4,5,28-28,SugarcaneClayey,4
1,27,69,65,Sandy,Millets,30,6,18,28-28,MilletsSandy,4
5,30,59,29,Red,Pulses,10,0,9,20-20,PulsesRed,3
6,27,62,53,Sandy,Paddy,26,15,22,28-28,PaddySandy,4
9,28,50,35,Red,Tobacco,25,12,16,20-20,TobaccoRed,3
...,...,...,...,...,...,...,...,...,...,...,...
749990,36,50,61,Loamy,Paddy,4,10,0,28-28,PaddyLoamy,4
749991,26,67,58,Sandy,Pulses,32,15,4,28-28,PulsesSandy,4
749992,37,54,53,Red,Wheat,14,4,18,20-20,WheatRed,3


In [7]:
df_prob = sc.read_prd('lr_e_1', index = df_train.index)

In [8]:
df_prd = pd.DataFrame(
    le.inverse_transform(np.argsort(-df_prob.values, axis=1)[:, :3].reshape(-1)).reshape(-1, 3),
    index = df_train.index
)

In [10]:
pd.crosstab(
    index = df_prd[0].astype(df_train[target].dtype), columns = df_train[target], margins = True, normalize = 'index'
)

Fertilizer_Name,28-28,17-17-17,10-26-26,DAP,20-20,14-35-14,Urea
0,,,,,,,
28-28,0.219142,0.131957,0.132701,0.120186,0.142276,0.132444,0.121294
17-17-17,0.135632,0.236740,0.136730,0.105392,0.133772,0.142309,0.109425
10-26-26,0.133711,0.133156,0.231546,0.112724,0.138019,0.141921,0.108923
DAP,0.135573,0.121490,0.129598,0.232505,0.136379,0.125804,0.118651
20-20,0.142860,0.130841,0.138224,0.120672,0.219976,0.134012,0.113415
14-35-14,0.135006,0.139202,0.141822,0.107897,0.133918,0.231032,0.111123
Urea,0.139701,0.125460,0.127245,0.122668,0.133053,0.127170,0.224702
All,0.148211,0.149937,0.151849,0.126480,0.147852,0.152581,0.123089
